Delete the DVH with no data as these complicate the load.

File always ends with two lines of white space

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
import os

In [2]:
def list_txt():
    files = os.listdir()   # return a list of files
    return [file for file in files if file.endswith('.txt')]

In [3]:
files = list_txt()
print(files)

['MSC5-DVH_data_export_ALLNEW.txt', 'MSC5-DVH_data_export_PLANSUM.txt', 'MSC8-DVH_data_export_ALLNEW.txt', 'MSC8-DVH_data_export_PLANSUM.txt']


In [4]:
file = files[0]
file

'MSC5-DVH_data_export_ALLNEW.txt'

In [5]:
with open(file, "r") as file_:
    my_file = [line for line in file_.readlines()]  # my_file is a list representation of the text file
file_.close()        

file_len = len(my_file)     # number of lines in the file, addd the 1 for indexing 
dvh_length = -1                # we will determine the length of the DVH
file_type = 'undeclared'      # will determine if a plansum or multi_plan
structures_indexs_list = []   # Get the structures
plans_list = []               # Populate a list of the plans in the file 

for i, line in enumerate(my_file):
    if line.startswith('Structure:'):
        structures_indexs_list.append(i)     # Get the index of all the structures

plans_list = [my_file[i+2].split(':')[-1].strip('\n').strip()  for i in structures_indexs_list]

if len(plans_list) is 1:      # If only a single DVH in the file then it's a plansum
    file_type = 'plansum'
    dvh_length = file_len - 2 - (structures_indexs_list[0] + 19)     # end - start of dvh
    DVH_data = my_file[structures_indexs_list[0]+19:structures_indexs_list[0]+19+dvh_length]   # get that DVH data in a list
    
    Dose_Gy    = [float(line.strip('\n').split()[0]) for line in DVH_data]
    Ratio_pct  = [float(line.strip('\n').split()[1]) for line in DVH_data]
    df = pd.DataFrame(index=Dose_Gy, data=Ratio_pct, columns=[plans_list[0]])
else:
    file_type = 'multi_plan' 
    dvh_length = file_len - 2 - (structures_indexs_list[-1] + 19)     # end - start of dvh
    
## to do
    
    

print('The file_type is %s' % file_type)
print('The length of a DVH is ' + str(dvh_length))

The file_type is multi_plan
The length of a DVH is 666


# Going to need this

In [6]:
for i, index in enumerate(structures_indexs_list):
    print(index)

65
752
771
1458
2145
2832
3519
4206
4893
5580
6267


In [7]:
DVH_data = my_file[structures_indexs_list[0]+19:structures_indexs_list[1]-2]

In [ ]:
Dose_Gy    = [float(line.strip('\n').split()[0]) for line in DVH_data]
Ratio_pct  = [100.0*(float(line.strip('\n').split()[2])/float(line.strip('\n').split()[2])) for line in DVH_data]

### Use for reference

In [ ]:
def Load_patient(file):  # file = 'Case1_AAA.txt'  string
    with open(file, "r") as file_:
        my_file = [line for line in file_.readlines()]  # my_file is a list representation of the text file
    file_.close()        
    file_len = len(my_file)                                # number of lines in the file
    patID = my_file[1].split(':')[-1].strip('\n').strip()
    planID = my_file[10].split(':')[-1].strip('\n').strip()
    
        
    ## Get the structures
    structures_indexs_list = []
    structures_names_list  = []
    for i, line in enumerate(my_file):
        if line.startswith('Structure:'):
            structures_indexs_list.append(i)  
            structures_names_list.append(line.split(':')[-1].strip('\n').strip())
    
    
    ##structures_names_list = ['PTV CHEST', 'Foramen'] # hard code to limit number of structures and prevent memory issues
    
    print(file + ' loaded \t patID:' + patID + ' PlanID:' + planID + ' and number of structures is ' + str(len(structures_names_list)))
    dose_index = np.linspace(0,100, 2001)  # New dose index in range 0 - 100 Gy in 0.05 Gy steps
    
    data = np.zeros((dose_index.shape[0], len(structures_names_list)))
    
    # iterate through all structures and place interpolated DVH data in matrix
    for i, index in enumerate(structures_indexs_list):
        start = structures_indexs_list[i]+18  # first line of DVH data
        if i < len(structures_indexs_list)-1:
            end = structures_indexs_list[i+1]-2  # find the last line of the DVH from the next index, BEWARE THE +1
        else:
            end = len(my_file)-2
        DVH_data = my_file[start:end]  # get list with data
            
        DVH_list = [line.split() for line in DVH_data]  # create list of lists split
        Rel_dose_pct, Dose_Gy, Ratio_pct = zip(*DVH_list) # unzip to 3 lists, they are strings so conver to float
        
        Ratio_pct = np.asarray(Ratio_pct, dtype=np.float32)
        Dose_Gy = np.asarray(Dose_Gy, dtype=np.float32)
        ## Now working with dose data
      
        f = interpolate.interp1d(x=Dose_Gy,y=Ratio_pct, bounds_error=False, fill_value=0)   # returns a linear interpolate function, fill values outside range wiwth 0 

        data[:,i] = f(dose_index)
    
    my_iterables = [[patID], [planID], structures_names_list]
    my_index = pd.MultiIndex.from_product(my_iterables, names = ['Patient ID', 'Plan ID', 'Structure'])

    df = pd.DataFrame(data.T, index = my_index)
    df = df.T
    df.index  = dose_index
    df.index.name = 'Dose (Gy)'
    return d